
# テーマ：日足とその日の初めの分足の足型が一致するかを分析する

# 仕様
* 東証上場銘柄（4,000社以上）を対象とする
  * https://www.jpx.co.jp/markets/statistics-equities/misc/01.html より銘柄一覧をダウンロードし、tickers.csvとして保管しておく。この際、銘柄コードの列名はtickerとする
* 指定した日数の日足と分足（デフォルトでは5分足）の株価をYahooから取得し、日足とその日の初めの分足の足型が一致するかを分析する。結果はresult.csvに出力する。result.csvの各列は以下の通り
  * match_rate：方向性が一致した割合
  * match_count：方向性が一致した数
  * whole_count：日足の数
  * average_volume：出来高の平均値（日足ベース）
  * average_price：株価の平均値（日足の終値ベース）
* Yahooから株価を取得する際、サーバー負荷をかけすぎないようにするため、一定秒数（デフォルトでは5秒）待ってから次の銘柄のデータをダウンロードする。そのため全銘柄ダウンロードするには6時間以上かかる
* 再ダウンロードの機会を減らすため、取得したJSONをファイルに出力（json_dataディレクトリ以下）

# 出来ていないこと
* ローカルファイル（保管したCSVファイル）を利用した分析

# 今回参考にしたもの
* https://note.com/aiduudia/n/na9ea4f90e255



# 1. Preparation

In [ ]:
# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'/'プログラム（株）'/日足と分足の関連分析

In [ ]:
# Hyper Dashのインストールとサインアップ
!pip install hyperdash
from hyperdash import monitor_cell
!hyperdash signup --github

# 2. Main

In [3]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta, timezone
from statistics import mean

# 定数、グローバル変数
RANGE = '60d' # 何日間のデータを取得するか
DAY_INTERVAL = '1d'
MINUTE_INTERVAL = '5d' # 分足は5分足をデフォルトとする
VOLUME_CRITERIA = 200000 # 出来高がこれより少ないものは無視する
INTERVAL_SECOND = 5 # この秒数待ってから次の銘柄のデータをダウンロードする

JST = timezone(timedelta(hours=+9), 'JST')

# Yahooから株価データを取得
def get_price_data_from_yahoo(ticker):
  json_day, json_minute = None, None

  # 日足取得
  response_day = requests.get('https://query1.finance.yahoo.com/v7/finance/chart/' \
                              + ticker + '?range=' + RANGE + '&interval=' + DAY_INTERVAL + '&indicators=quote&includeTimestamps=true')
  
  if 200 <= response_day.status_code < 300:
    json_day = response_day.json()
    
    # 平均出来高が基準未満であれば以降の処理をスキップ
    volume = 0
    average_volume = 0
    if 'volume' in json_day['chart']['result'][0]['indicators']['quote'][0]:
      volume = json_day['chart']['result'][0]['indicators']['quote'][0]['volume']
      average_volume = mean(d for d in volume if d is not None)
    
    if volume is None or average_volume < VOLUME_CRITERIA:
      return None, None
    
    # データを再利用可能にするため、JSONファイルを保管
    start_date = extract_date(json_day['chart']['result'][0]['timestamp'][0])
    end_date = extract_date(json_day['chart']['result'][0]['timestamp'][-1])
    save_json(json_day, r'./json_data/' + f'{ticker}_day_{start_date}_{end_date}.json')

  # 分足取得
  response_minute = requests.get('https://query1.finance.yahoo.com/v7/finance/chart/' \
                              + ticker + '?range=' + RANGE + '&interval=' + MINUTE_INTERVAL + '&indicators=quote&includeTimestamps=true')
  
  if 200 <= response_minute.status_code < 300:
    json_minute = response_minute.json()

    # データを再利用可能にするため、JSONファイルを保管
    start_date = extract_date(json_minute['chart']['result'][0]['timestamp'][0])
    end_date = extract_date(json_minute['chart']['result'][0]['timestamp'][-1])
    save_json(json_minute, r'./json_data/' + f'{ticker}_minute_{start_date}_{end_date}.json')

  return json_day, json_minute

# タイムスタンプから日付を取得
def extract_date(timestamp):
  date_and_time = str(datetime.fromtimestamp(timestamp))
  date, _ = date_and_time.split()
  return date

# JSONファイル保存
def save_json(json_data, file_path):
  with open(file_path, 'w') as f:
    json.dump(json_data, f, indent=2)

# 株価データ（JSON）をデータフレームに変換
def convert_to_dataframe(json_day, json_minute):
  # 日足
  if json_day['chart']['result'] == None or 'timestamp' not in json_day['chart']['result'][0]:
    return None, None
  df_day = pd.DataFrame()
  df_day['timestamp'] = list(map(datetime.fromtimestamp, json_day['chart']['result'][0]['timestamp'], [JST]*len(json_day['chart']['result'][0]['timestamp'])))
  df_day['open'] = json_day['chart']['result'][0]['indicators']['quote'][0]['open']
  df_day['low'] = json_day['chart']['result'][0]['indicators']['quote'][0]['low']
  df_day['high'] = json_day['chart']['result'][0]['indicators']['quote'][0]['high']
  df_day['close'] = json_day['chart']['result'][0]['indicators']['quote'][0]['close']
  df_day['volume'] = json_day['chart']['result'][0]['indicators']['quote'][0]['volume']
  df_day['positive'] = df_day['open'] < df_day['close']
  df_day['minute_positive'] = None

  # 5分足
  if json_minute['chart']['result'] == None or 'timestamp' not in json_minute['chart']['result'][0]:
    return None, None
  df_minute = pd.DataFrame()
  df_minute['timestamp'] = list(map(datetime.fromtimestamp, json_minute['chart']['result'][0]['timestamp'], [JST]*len(json_minute['chart']['result'][0]['timestamp'])))
  df_minute['open'] = json_minute['chart']['result'][0]['indicators']['quote'][0]['open']
  df_minute['low'] = json_minute['chart']['result'][0]['indicators']['quote'][0]['low']
  df_minute['high'] = json_minute['chart']['result'][0]['indicators']['quote'][0]['high']
  df_minute['close'] = json_minute['chart']['result'][0]['indicators']['quote'][0]['close']
  df_minute['volume'] = json_minute['chart']['result'][0]['indicators']['quote'][0]['volume']

  return df_day, df_minute

# 日足の方向性（陽線/陰線）とその日の初めの5分足の方向性が一致している割合を算出
def analyze_direction(df_day, df_minute, index, df_result):
  # その日の初めの5分足が陽線かどうかをチェック
  for row_day in df_day.itertuples():
    for row_minute in df_minute.itertuples():
      if row_day.timestamp.date() == row_minute.timestamp.date():
        # 出来高0の場合は無視する
        if row_minute.volume <= 0:
          continue
        df_day.at[row_day[0], 'minute_positive'] = row_minute.open < row_minute.close
        break

  df_day['same_direction'] = [1 if r.positive == r.minute_positive else 0 for r in df_day.itertuples()]

  df_result.at[index, 'match_rate'] = df_day['same_direction'].sum() / df_day['same_direction'].count()
  df_result.at[index, 'match_count'] = df_day['same_direction'].sum()
  df_result.at[index, 'whole_count'] = df_day['same_direction'].count()
  df_result.at[index, 'average_volume'] = df_day['volume'].mean()
  df_result.at[index, 'average_price'] = df_day['close'].mean()

# 一連の処理
def analyzer(ticker, df_result, index):
  json_day, json_minute = get_price_data_from_yahoo(ticker)
  if json_day is None or json_minute is None:
    return

  df_day, df_minute = convert_to_dataframe(json_day, json_minute)
  if df_day is None or df_minute is None:
    return

  analyze_direction(df_day, df_minute, index, df_result)

In [ ]:
%%monitor_cell 'Analyzer'

import codecs
import time
from IPython.display import display

#df_result = pd.read_csv('./tickers.csv')
# UTF-8のエラー対応のため上のコードの代わりに次のコードを実行し、銘柄コード一覧を取得
with codecs.open('./tickers.csv', 'r', 'UTF-8', 'ignore') as file:
  df_result = pd.read_table(file, delimiter=',')

# 東証前提のため銘柄コードの末尾に.Tを付与
df_result['ticker'] = list(map(str, df_result['ticker']))
df_result['ticker'] = df_result['ticker'] + '.T'

start = time.time()

for row in df_result.itertuples():
  analyzer(row.ticker, df_result, row[0])
  time.sleep(INTERVAL_SECOND)
  if row[0] % 100 == 99:
  # if row[0] % 10 == 3: # テスト用コード
    process_time = time.time() - start
    print(f'{str(row[0]+1)}社目のデータ取得後の結果（これまで{str(round(process_time/60, 1))}分経過）')
    display(df_result.sort_values('match_rate', ascending=False))

df_result = df_result.dropna(how='any')
if 'match_count' in df_result:
  df_result['match_count'] = list(map(int, df_result['match_count']))
if 'whole_count' in df_result:
  df_result['whole_count'] = list(map(int, df_result['whole_count']))

df_result.to_csv('./result.csv')

# 3. Code Snippet

In [ ]:
# デバッガー
%debug

In [ ]:
# デバッグするときは次のコードをブレイクポイントに設定
from IPython.core.debugger import Pdb; Pdb().set_trace()

# ノートブックの稼働時間を調べたいときは次のコードを別ノートブックで実行
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 / 60 / 60 "h)"}'